In [1]:
import os
import math

import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageDraw

from scorefield.models.ddpm.denoising_diffusion import Unet
from scorefield.models.rrt.rrt import RRTStar
from scorefield.utils.rl_utils import load_config
from scorefield.utils.utils import (
    gen_goals, overlay_goal, overlay_multiple, combine_objects, overlay_images,
    overlay_goal_agent, overlay_goals_agent, log_num_check,
    draw_obstacles_pil, convert_to_obstacle_masks,
    randgen_obstacle_masks, draw_obstacles_pixel,
    vector_field, clip_vectors
)
from scorefield.utils.diffusion_utils import bilinear_interpolate, bilinear_interpolate_samples

import matplotlib.pyplot as plt
import itertools
from typing import Optional
import shutil


# Args
config_dir = "./scorefield/configs/rrt.yaml"
args = load_config(config_dir)
device = args['device']

bg = Image.open('assets/toy_exp/background0.png')
wastes = []
wastes.append(Image.open('assets/toy_exp/waste0.png'))
# wastes.append(Image.open('assets/toy_exp/waste4.png'))
# wastes.append(Image.open('assets/toy_exp/waste5.png'))

img_size = args['image_size']
delta_dist = args['delta_dist']
radius = args['radius']
time_steps = args['time_steps']

goal_bounds = args['goal_bounds']
agent_bounds = args['agent_bounds']
obstacle_pos = args['obstacles']

iterations = args['iterations']
train_lr = args['train_lr']
batch_size = args['batch_size']

/home/mlcs_edf/mambaforge/envs/diffusion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Unet2D(Unet):
    def __init__(
        self, 
        dim, 
        out_dim, 
        dim_mults=(1, 2, 4, 8),
    ):
        super().__init__(dim=dim, out_dim=out_dim, dim_mults=dim_mults)

    def forward(self, obs, t, x_t:Optional[torch.Tensor]=None):
        score_map = super().forward(obs, t)
        score = bilinear_interpolate_samples(score_map, x_t)    # output: (B,2)
        return score

    
model = Unet2D(
    dim=img_size,
    out_dim = 2,
    dim_mults = (1, 2, 4, 8),
).to(device)

rrt = RRTStar(
    image_size=img_size,
    time_steps=time_steps,
    delta_dist=delta_dist,
    radius=radius,
    device=device,
)

optim = torch.optim.Adam(params=model.parameters(), lr=train_lr)

In [3]:
# Train with random single goal

# writer = SummaryWriter()
# shutil.copy('./scorefield/configs/rrt.yaml', writer.log_dir)

for iters in tqdm(range(iterations)):
    obstacle_masks = randgen_obstacle_masks(batch_size, img_size)
    background = draw_obstacles_pixel(bg, obstacle_masks)
    goals = gen_goals(goal_bounds, batch_size, img_size, obstacles=obstacle_masks)
    goal = (torch.rand(batch_size, 1, 2, device=device, dtype=torch.float32) * 0.2 - 0.1) * 0.1 + goals
    obs = overlay_goal(background, img_size, wastes, goal)
    initials = gen_goals(goal_bounds, batch_size, img_size, obstacles=obstacle_masks)
    
    optim.zero_grad()
    
    losses = []
    with torch.no_grad():
        rrt_path = rrt.plan(initials, goal, obstacle_masks)
    rand_indices = rrt.random_sample()

    pred = model(obs, torch.tensor(rand_indices, device=device), initials)

    square_diff = torch.sum((pred - torch.tensor(rrt_path[rand_indices+1]))**2, dim=-1)
    loss = square_diff.mean()
    loss.backward()
    optim.step()
    
    print(loss.item())

#     if iters == iterations // 2:
#         torch.save(model.state_dict(), os.path.join(writer.log_dir, 'model_params_half.pt'))

# torch.save(model.state_dict(), os.path.join(writer.log_dir, 'model_params.pt'))
            
# writer.close()


  0%|          | 0/6000 [00:02<?, ?it/s]


TypeError: object of type 'NoneType' has no len()

In [5]:
for b in range(len(rrt_path)):
    print(len(rrt_path[b]))

60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60


In [6]:
rand_indices

[37, 41, 42, 11, 7, 42, 18, 28, 2, 2, 29, 21, 51, 7, 25, 15, 37, 20]